## Compute Average Noise Level per Path Based on Noise Data from Intersected Polygons

This script calculates the average **noise level** for each path based on the **noise polygons** it intersects. Each noise polygon contains noise level data in dbs, which was measured in Ljubljana.

### Features
- **Path-Based Noise Analysis**: The script evaluates the noise levels along different paths by averaging the noise values of the polygons they pass through.
- **Polygon Intersection**: Uses **shapely** to efficiently compute intersections between paths and noise polygons to calculate noise levels.
- **Flexible Input Format**: Takes in two GeoJSON files — one with noise polygon data (containing noise levels) and another with a set of paths represented by their geographic coordinates and lengths.

### Inputs
- **Noise Data (GeoJSON)**:
  - Contains geographic polygons with noise levels, representing areas of varying noise intensity, downloaded from https://data.noise-planet.org/noisecapture/.
- **Paths Data (GeoJSON)**:
  - A set of 25 paths with defined geometric lines and associated attributes like path length in meters to be assessed and compared based on noise.

### Output
- For each path, the script returns the **average noise level** based on the noise values of all polygons it intersects, providing insights into noise exposure along the route.

### Data Source
- **Environmental Noise Data**  
  - **Source**: [Data nise-planet] (https://data.noise-planet.org/noisecapture/)
  - **Format**: Noise data is pre-processed into GeoJSON format, with each polygon representing a specific noise zone and its corresponding level.


In [ ]:
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString

# Load your geojson data (noise data and path data)
# Path to your GeoJSON files
noise_file_path = '/Slovenia_Osrednjeslovenska_Ljubljana.tracks.geojson'
path_file_path = '/paths_test_json.json'

# Read the GeoJSON files
noise_gdf = gpd.read_file(noise_file_path)
path_gdf = gpd.read_file(path_file_path)

# Function to check if there is a noise data point for the path and compute the average noise
def check_noise_data_for_path(path, noise_gdf, path_length):
    total_noise = 0
    count = 0
    
    # Iterate over the noise polygons and check for intersection with the path
    for idx, noise_polygon in enumerate(noise_gdf['geometry']):
        if path.intersects(noise_polygon):
            total_noise += noise_gdf.iloc[idx]['noise_level']
            count += 1
    
    # Check if the average distance per noise point is smaller than 500 meters
    if count > 0 and path_length / count < 500:
        # Compute the average noise level if the condition is met
        return total_noise / count
    else:
        # If no noise points or the average distance per point is <= 500 meters, return None
        return None

# Compute the average noise level for all paths
path_noise_data = []

for idx, path in path_gdf.iterrows():
    path_length = path['length_m']  # Get the path length from the JSON
    # Check if the path has noise data points at intervals greater than 500 meters        
    average_noise = check_noise_data_for_path(path['geometry'], noise_gdf, path_length)
    if average_noise is not None:  # Only add paths with valid average noise levels
        path_noise_data.append((idx, average_noise))  # Save index and noise level

# Sort paths by average noise level (quietest first)
path_noise_data_sorted = sorted(path_noise_data, key=lambda x: x[1])  # Sort by average noise level

# Get the indexes of the quietest 3 paths
quietest_3_indexes = [item[0] for item in path_noise_data_sorted[:3]]

# Subselect the quietest 3 paths using their indexes
quietest_3_paths_gdf = path_gdf.loc[quietest_3_indexes]

# Print the quietest 3 paths GeoDataFrame
print(quietest_3_paths_gdf)



Skipping field tags: unsupported OGR type: 5
Skipping field origin_coords: unsupported OGR type: 3
Skipping field destination_coords: unsupported OGR type: 3


  id  path_num    length_m                                        coordinates   
1  1         2  794.769771  [ [ 14.5054488, 46.051100699999999 ], [ 14.505...  \
2  2         3  795.679623  [ [ 14.5054488, 46.051100699999999 ], [ 14.505...   
3  3         4  795.741988  [ [ 14.5054488, 46.051100699999999 ], [ 14.505...   

                                            geometry  
1  LINESTRING (14.50545 46.0511, 14.50578 46.0513...  
2  LINESTRING (14.50545 46.0511, 14.50578 46.0513...  
3  LINESTRING (14.50545 46.0511, 14.50578 46.0513...  


In [2]:
average_noise_data_sorted = [item[1] for item in path_noise_data_sorted]
average_noise_data_sorted 

[73.83930733019498,
 73.83930733019498,
 73.83930733019498,
 73.83930733019498,
 73.83930733019498,
 73.83930733019498,
 73.83930733019498,
 73.83930733019498,
 73.83930733019498,
 73.91092517006796]